# Prediction with ForceRegression model

In [ ]:
# %load imports.py
%load_ext autoreload
%autoreload 2
%reload_kedro
%config Completer.use_jedi = False  ## (To fix autocomplete)
import pandas as pd
from src.models.vmm import ModelSimulator
import matplotlib.pyplot as plt
from src.visualization.plot import track_plots, plot, captive_plot
import kedro
import numpy as np
import os.path
import anyconfig

import matplotlib
matplotlib.rcParams["figure.figsize"] = (15,4)
from src.symbols import *

# Read configs:
conf_path = os.path.join("../conf/base/")
runs_globals_path = os.path.join(
    conf_path,
    "runs_globals.yml",
)

runs_globals = anyconfig.load(runs_globals_path)
model_test_ids = runs_globals["model_test_ids"]

join_globals_path = os.path.join(
    conf_path,
    "join_globals.yml",
)

joins = runs_globals["joins"]
join_runs_dict = anyconfig.load(join_globals_path)

globals_path = os.path.join(
    conf_path,
    "globals.yml",
)
global_variables = anyconfig.load(globals_path)



vmms = global_variables["vmms"]
only_joined = global_variables[
    "only_joined"
]  # (regress/predict with only models from joined runs)S

In [ ]:
#id = "22773"
id = "22774"
vmm_name = "vmm_martin"
#vmm_name = "vmm_martins_simple"

df = catalog.load(f"{id}.data")
df_smooth = catalog.load(f"{id}.data_ek_smooth")
model = catalog.load(f"{vmm_name}.force_regression.model")
regression = catalog.load(f"{vmm_name}.force_regression.regression")
df_VCT = catalog.load("force_regression.data_scaled")
ship_data = catalog.load("ship_data")

model_motion_regression = catalog.load(f"{vmm_name}.motion_regression.joined.model")


motion_regression = catalog.load(f"{ vmm_name }.motion_regression.joined.regression")
force_regression = catalog.load(f"{ vmm_name }.force_regression.regression")
#keys_forces = ['fx','fy','fz']
#scale_factor=ship_data['scale_factor']
#forces=df_VCT[keys_forces]
#df_VCT[keys_forces] = scale_force_to_model_scale(forces=forces, scale_factor=scale_factor, **df_VCT)
#keys_moments = ['mx','my','mz']
#df_VCT[keys_moments] = scale_force_to_model_scale(df_VCT[keys_moments], scale_factor=scale_factor, **df_VCT)


In [ ]:
motion_regression.parameters

In [ ]:
motion_regression.model_X.summary()

In [ ]:
force_regression.parameters

In [ ]:
force_regression.model_Y.summary()

In [ ]:
force_regression.model_Y.cov_HC0

In [ ]:
force_regression.model_Y.bse

In [ ]:
from scipy.stats import norm, multivariate_normal

In [ ]:
rv = norm(loc=force_regression.model_Y.params['Yv'], scale=force_regression.model_Y.bse['Yv'])
rv.interval(alpha=0.95)

In [ ]:
bse = force_regression.model_X.bse

means = force_regression.model_X.params
means = means.append(force_regression.model_Y.params)
means = means.append(force_regression.model_N.params)

stds = force_regression.model_X.bse
stds = stds.append(force_regression.model_Y.bse)
stds = stds.append(force_regression.model_N.bse)

mask = stds>10**-10
means=means[mask].copy()
stds=stds[mask].copy()

In [ ]:
cov = np.diag(stds.values**2)
rv = multivariate_normal(mean=means, cov=cov)

In [ ]:
df_parameter_variation = pd.DataFrame(data=rv.rvs(10), columns=means.index)

In [ ]:
df_parameter_variation.head()

In [ ]:
models = {}
results = {}
#for index, parameters in df_parameter_variation.iterrows():
#    
#    model_ = model.copy()
#    model_.parameters.update(parameters)
#    
#    try:
#        result = model_.simulate(df_=df_smooth)
#    except:
#        continue
#        
#    models[index] = model_
#    results[index] = result.result
#    

In [ ]:
#dataframes = {
#'model test' : df_smooth,
#}
#
#dataframes.update(results)
#
#styles = {key:{'style':'r-','alpha':0.3} for key in dataframes}
#styles['model test'] = {'style':'b-', 'alpha':1, 'zorder':-10}
#
#fig,ax=plt.subplots()
#track_plots(dataframes=dataframes, lpp=model.ship_parameters['L'], beam=model.ship_parameters['B'], ax=ax, plot_boats=False, styles=styles);
#
#plot(dataframes, keys=result.result.columns);

In [ ]:
parameters = force_regression.parameters
parameters['yerr'] = (parameters['$conf_{higher}$'] - parameters['$conf_{lower}$']).abs()
parameters.plot.bar(y='regressed', yerr=parameters['yerr'])

In [ ]:
parameters.plot.bar(y='$P_{value}$')

In [ ]:
df_VCT[['u','v','V','r','fx','fy','mz']].abs().max()

In [ ]:
df_parameters = pd.DataFrame()
df_parameters['force regression'] = model.parameters
df_parameters['motion regression'] = model_motion_regression.parameters
df_parameters

In [ ]:
drops = [key for key in df_parameters.index if 'dot' in key]
drops.append('Xthrust')
df_parameters_plot = df_parameters.drop(index=drops)
df_parameters_plot.plot(kind='bar')

In [ ]:
#model.parameters['Xu'] = -0.000737*1.7
result = model.simulate(df_=df_smooth)
result2 = model_motion_regression.simulate(df_=df_smooth)

In [ ]:
#model_hybrid = model.copy()
#steals = ['Xvr']
#
#for steal in steals:
#    model_hybrid.parameters[steal] = model_motion_regression.parameters[steal]
#    
#result_hybrid = model_hybrid.simulate(df_=df_smooth)

In [ ]:
dataframes = {
'model test' : df_smooth,
'simulation (force regression)' : result.result,  
'simulation (motion regression)' : result2.result,  
#'simulation (hybrid)' : result_hybrid.result,

}

fig,ax=plt.subplots()
track_plots(dataframes=dataframes, lpp=model.ship_parameters['L'], beam=model.ship_parameters['B'], ax=ax, plot_boats=False);

plot(dataframes, keys=result.result.columns);

In [ ]:
catalog.load(f"{ vmm_name }.motion_regression.joined.{id}.simulation_accuracy")

In [ ]:
regressions = global_variables['regressions']

regression_names = ['motion_regression','force_regression']

df_accuracies = pd.DataFrame()

for vmm_name in vmms:
    for model_test_id in model_test_ids:
        
        file_paths = []
        file_paths.append(f"{ vmm_name }.motion_regression.joined.{model_test_id}.simulation_accuracy")
        file_paths.append(f"{ vmm_name }.force_regression.{model_test_id}.simulation_accuracy")
        
        
        for regression,file_path in zip(regression_names, file_paths):
            try:
                accuracy = catalog.load(file_path)
            except:
                continue
            
            accuracy = pd.Series(accuracy)
            accuracy['total'] = accuracy.mean()
            
            accuracy['vmm'] = vmm_name
            accuracy['regression'] = regression
            accuracy['model test'] = model_test_id
            
            df_accuracies = df_accuracies.append(accuracy, ignore_index=True)
        




In [ ]:
df_compare = df_accuracies.groupby(by=['vmm','regression']).mean().sort_values(by='total', ascending=True)
df_compare

In [ ]:
df_compare['total'].plot.bar()

In [ ]:
df_compare.reset_index().pivot(index=['vmm'], columns=['regression'], values='total').plot.bar();